In [ ]:
# Markdown content
markdown_notes = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes"""

In [ ]:
# !lsof -i:8080

COMMAND PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
node      7 root   21u  IPv6  19658      0t0  TCP *:8080 (LISTEN)
node      7 root   26u  IPv6  26842      0t0  TCP e0d60b048021:8080->172.28.0.1:35198 (ESTABLISHED)
node      7 root   28u  IPv6  28081      0t0  TCP e0d60b048021:8080->172.28.0.1:35210 (ESTABLISHED)
node      7 root   30u  IPv6  28119      0t0  TCP e0d60b048021:8080->172.28.0.1:35220 (ESTABLISHED)


In [ ]:
import re
from google.colab import auth
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

# Authenticate and initialize Google Docs API
SCOPES = ['https://www.googleapis.com/auth/documents']
CLIENT_SECRET_FILE = '/content/client_secret.json'

def authenticate_with_google():
    auth.authenticate_user()
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
    creds = flow.run_local_server(port=0)
    service = build('docs', 'v1', credentials=creds)
    return service

def create_google_doc(service, title):
    doc = service.documents().create(body={'title': title}).execute()
    return doc['documentId']

# Parse markdown content
def parse_markdown(md_content):
    lines = md_content.split('\n')
    structured_content = []
    bullet_indent = 0  # Tracks bullet indentation

    for line in lines:
        if line.startswith('# '):
            structured_content.append({'type': 'heading_1', 'text': line[2:]})
        elif line.startswith('## '):
            structured_content.append({'type': 'heading_2', 'text': line[3:]})
        elif line.startswith('### '):
            structured_content.append({'type': 'heading_3', 'text': line[4:]})
        elif line.startswith('- [ ]'):
            structured_content.append({'type': 'checkbox', 'text': line[6:]})
        elif line.startswith('- '):
            structured_content.append({'type': 'bullet', 'text': line[2:], 'indent': bullet_indent})
        elif line.strip() == "":
            bullet_indent = 0  # Reset indentation on empty lines
        else:
            structured_content.append({'type': 'text', 'text': line.strip()})

        if line.startswith("  "):  # Detect nested bullet points
            bullet_indent += 1

    return structured_content

# Converts parsed content into Google Docs API requests
def write_to_google_doc(service, document_id, content):
    try:
      requests = []
      index = 1  # Start writing at index 1

      for item in content:
          text = item['text'] + '\n'

          if item['type'] == 'heading_1':
              requests.append({
                  'insertText': {'location': {'index': index}, 'text': text}
              })
              requests.append({
                  'updateParagraphStyle': {
                      'range': {'startIndex': index, 'endIndex': index + len(text)},
                      'paragraphStyle': {'namedStyleType': 'HEADING_1'},
                      'fields': 'namedStyleType'
                  }
              })

          elif item['type'] == 'heading_2':
              requests.append({
                  'insertText': {'location': {'index': index}, 'text': text}
              })
              requests.append({
                  'updateParagraphStyle': {
                      'range': {'startIndex': index, 'endIndex': index + len(text)},
                      'paragraphStyle': {'namedStyleType': 'HEADING_2'},
                      'fields': 'namedStyleType'
                  }
              })

          elif item['type'] == 'heading_3':
              requests.append({
                  'insertText': {'location': {'index': index}, 'text': text}
              })
              requests.append({
                  'updateParagraphStyle': {
                      'range': {'startIndex': index, 'endIndex': index + len(text)},
                      'paragraphStyle': {'namedStyleType': 'HEADING_3'},
                      'fields': 'namedStyleType'
                  }
              })

          elif item['type'] == 'checkbox':
              requests.append({
                  'insertText': {'location': {'index': index}, 'text': text}
              })
              requests.append({
                  'createParagraphBullets': {
                      'range': {'startIndex': index, 'endIndex': index + len(text)},
                      'bulletPreset': 'CHECKBOX'
                  }
              })

          elif item['type'] == 'bullet':
              requests.append({
                  'insertText': {'location': {'index': index}, 'text': text}
              })
              requests.append({
                  'createParagraphBullets': {
                      'range': {'startIndex': index, 'endIndex': index + len(text)},
                      'bulletPreset': 'BULLET_DISC'
                  }
              })

          elif item['type'] == 'text':
              requests.append({
                  'insertText': {'location': {'index': index}, 'text': text}
              })

          # Handle @mentions (bold formatting)
          if '@' in text:
              mention_start = text.index('@') + index
              mention_end = mention_start + len(text.split('@')[1].split()[0])
              requests.append({
                  'updateTextStyle': {
                      'range': {'startIndex': mention_start, 'endIndex': mention_end},
                      'textStyle': {'bold': True, 'foregroundColor': {'color': {'rgbColor': {'red': 0, 'green': 0, 'blue': 1}}}},
                      'fields': 'bold,foregroundColor'
                  }
              })

          index += len(text)

      service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()

    except Exception as e:
      print(f"An error occurred: {e}")

# Main script
def main():

    service = authenticate_with_google()
    if not service:
        print("Authentication failed. Exiting.")
        return

    doc_id = create_google_doc(service, "Product Team Sync - May 15, 2023")
    content = parse_markdown(markdown_notes)
    write_to_google_doc(service, doc_id, content)
    print(f"Document created successfully: https://docs.google.com/document/d/{doc_id}")

if __name__ == "__main__":
    main()
